In [23]:
# Package installs
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install seaborn
%pip install scikit-learn
%pip install cartopy
%pip install keras
%pip install keras-tuner
# you may have to install in console using pip install tensorflow --user or pip install tensorflow --upgrade
%pip install tensorflow
%pip install basketball-reference-scraper

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated pac

In [4]:
%pip install nba_api

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ------------- -------------------------- 5.2/15.5 MB 26.5 MB/s eta 0:00:01
   ------------------------- -------------- 9.7/15.5 MB 24.1 MB/s eta 0:00:01
   ------------------------------------- -- 14.7/15.5 MB 26.4 MB/s eta 0:00:01
   ---------------------------------------- 15.5/15.5 MB 21.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import datetime
import time
import basketball_reference_scraper
# Modeling tools
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
import keras_tuner as kt
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import OutputType
from datetime import date
from nba_api.stats.endpoints import leaguegamelog

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.metrics import MeanAbsoluteError
import keras_tuner as kt
import joblib

In [ ]:
player_stats = client.players_season_totals(season_end_year=2025)
print(player_stats)  # List of all players' season stats

[{'slug': 'gilgesh01', 'name': 'Shai Gilgeous-Alexander', 'positions': [<Position.POINT_GUARD: 'POINT GUARD'>], 'age': 26, 'team': <Team.OKLAHOMA_CITY_THUNDER: 'OKLAHOMA CITY THUNDER'>, 'games_played': 70, 'games_started': 70, 'minutes_played': 2402, 'made_field_goals': 795, 'attempted_field_goals': 1522, 'made_three_point_field_goals': 148, 'attempted_three_point_field_goals': 400, 'made_free_throws': 563, 'attempted_free_throws': 625, 'offensive_rebounds': 60, 'defensive_rebounds': 293, 'assists': 440, 'steals': 123, 'blocks': 71, 'turnovers': 175, 'personal_fouls': 153, 'points': 2301}, {'slug': 'edwaran01', 'name': 'Anthony Edwards', 'positions': [<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>], 'age': 23, 'team': <Team.MINNESOTA_TIMBERWOLVES: 'MINNESOTA TIMBERWOLVES'>, 'games_played': 70, 'games_started': 70, 'minutes_played': 2539, 'made_field_goals': 628, 'attempted_field_goals': 1425, 'made_three_point_field_goals': 279, 'attempted_three_point_field_goals': 706, 'made_free_throws'

In [ ]:
df1 = pd.DataFrame(player_stats)

In [ ]:
df2 = pd.DataFrame(player_stats)

# Calculate per-game averages
df2["points_per_game"] = df2["points"] / df2["games_played"]
df2["assists_per_game"] = df2["assists"] / df2["games_played"]
df2["total_rebounds"] = df2["offensive_rebounds"] + df2["defensive_rebounds"]
df2["rebounds_per_game"] = df2["total_rebounds"] / df2["games_played"]
df2["steals_per_game"] = df2["steals"] / df2["games_played"]
df2["blocks_per_game"] = df2["blocks"] / df2["games_played"]
df2["turnovers_per_game"] = df2["turnovers"] / df2["games_played"]

# Display only relevant columns
df2 = df2[["name", "points_per_game", "assists_per_game", "rebounds_per_game","blocks_per_game" , "turnovers_per_game"]]

In [ ]:
df2.to_csv("average_stats.csv", index=False)

In [ ]:
all_games_2025 = client.season_schedule(season_end_year=2025)
df_2025 = pd.DataFrame(all_games_2025)

csv_filename = "NBA_2025_schedule.csv"
teamlist = []
for team in df_2025['away_team']:
    add = True
    for i in teamlist:
        if team == i:
            add = False
    if(add):        
        teamlist.append(team)

df_tlist =pd.DataFrame(teamlist)
df_tlist.to_csv("ListOfTeams.csv",index = False)
df_2025.to_csv( "NBA_2025_schedule.csv", index = False)      

In [ ]:
df = pd.read_csv("NBA_2025_schedule.csv")


df = df.dropna(subset=['away_team_score'])

# Save the cleaned DataFrame
df.to_csv("NBA_2025_cleaned_schedule.csv", index=False)
        
        


'''df['away_team'] = df['away_team'].str.replace("TEAM.", '', regex=True)
df['home_team'] = df['home_team'].str.replace("TEAM.", "Bruh", regex=True)
df.to_csv( "NBA_20256_schedule.csv", index = False)'''      

'df[\'away_team\'] = df[\'away_team\'].str.replace("TEAM.", \'\', regex=True)\ndf[\'home_team\'] = df[\'home_team\'].str.replace("TEAM.", "Bruh", regex=True)\ndf.to_csv( "NBA_20256_schedule.csv", index = False)'

In [ ]:
df = pd.read_csv("NBA_2025_cleaned_schedule.csv")

df['away_team'] = df['away_team'].str.replace('Team.','',)
df['home_team'] = df['home_team'].str.replace('Team.','',)

df.to_csv("NBA_2025_OFFICIAL_TEAM_SCHEDULE.csv", index=False)

In [ ]:
df = pd.read_csv("NBA_2025_OFFICIAL_TEAM_SCHEDULE.csv")
df1 = pd.read_csv("ListOfTeams.csv")
df['TEAMS'] = df1

df.to_csv("NBA_2025_OFFICIAL_TEAM_SCHEDULE.csv", index=False)

In [ ]:

df = pd.read_csv("NBA_2025_OFFICIAL_TEAM_SCHEDULE.csv")
calcdf = []

for i in range(len(df)):
    home_score = df.at[i, 'home_team_score']
    away_score = df.at[i, 'away_team_score']
    home_team = df.at[i, 'home_team']
    away_team = df.at[i, 'away_team']

    home_won = 1 if home_score > away_score else 0
    away_won = 1 if away_score > home_score else 0

    # Append home team results
    calcdf.append({
        'TEAM': home_team,
        'OPPONENT': away_team,
        'TEAM_HOME': 1,  # 1 = Home Game
        'WIN': home_won,
        'TEAM_SCORE': home_score,
        'OPPONENT_SCORE': away_score
    })

    # Append away team results
    calcdf.append({
        'TEAM': away_team,
        'OPPONENT': home_team,
        'TEAM_HOME': 0,  # 0 = Away Game
        'WIN': away_won,
        'TEAM_SCORE': away_score,
        'OPPONENT_SCORE': home_score
    })

# Convert to DataFrame
dflonger = pd.DataFrame(calcdf)

# Group and summarize matchups properly
dfsummary = dflonger.groupby(['TEAM', 'OPPONENT', 'TEAM_HOME']).agg(
    games_played=('WIN', 'size'),
    wins=('WIN', 'sum'),
    total_points_scored=('TEAM_SCORE', 'sum'),
    total_points_allowed=('OPPONENT_SCORE', 'sum'),
    avg_points_scored=('TEAM_SCORE', 'mean'),
    avg_points_allowed=('OPPONENT_SCORE', 'mean')
).reset_index()

# Save to CSV
dfsummary.to_csv("NBA_Team_Performance.csv", index=False)

# Display summary
print(dfsummary)

                    TEAM           OPPONENT  TEAM_HOME  games_played  wins  \
0          ATLANTA_HAWKS     BOSTON_CELTICS          0             2     2   
1          ATLANTA_HAWKS     BOSTON_CELTICS          1             1     0   
2          ATLANTA_HAWKS      BROOKLYN_NETS          0             1     0   
3          ATLANTA_HAWKS      BROOKLYN_NETS          1             1     1   
4          ATLANTA_HAWKS  CHARLOTTE_HORNETS          0             2     2   
...                  ...                ...        ...           ...   ...   
1615  WASHINGTON_WIZARDS  SAN_ANTONIO_SPURS          1             1     0   
1616  WASHINGTON_WIZARDS    TORONTO_RAPTORS          0             2     1   
1617  WASHINGTON_WIZARDS    TORONTO_RAPTORS          1             2     0   
1618  WASHINGTON_WIZARDS          UTAH_JAZZ          0             1     0   
1619  WASHINGTON_WIZARDS          UTAH_JAZZ          1             1     1   

      total_points_scored  total_points_allowed  avg_points_sco

In [ ]:
start_date = datetime.date(2023, 10, 24)
end_date = datetime.date(2024, 4, 14)

current_date = start_date
all_box_scores = []

while current_date <= end_date:
    try:
        print(f"Getting box scores for {current_date}")
        day_stats = client.player_box_scores(
            day=current_date.day,
            month=current_date.month,
            year=current_date.year
        )
        for entry in day_stats:
            entry['date'] = current_date  # include game date
            all_box_scores.append(entry)
        time.sleep(1)  # be nice to the server
    except Exception as e:
        print(f"Failed to get data for {current_date}: {e}")
    current_date += datetime.timedelta(days=1)

# Convert to DataFrame
df = pd.DataFrame(all_box_scores)
df.to_csv("full_season_box_scores.csv", index=False)
print("✅ Done! Saved to full_season_box_scores.csv")

Getting box scores for 2023-10-24
Failed to get data for 2023-10-24: 429 Client Error: Too Many Requests for url: https://www.basketball-reference.com/friv/dailyleaders.cgi?month=10&day=24&year=2023
Getting box scores for 2023-10-25
Failed to get data for 2023-10-25: 429 Client Error: Too Many Requests for url: https://www.basketball-reference.com/friv/dailyleaders.cgi?month=10&day=25&year=2023
Getting box scores for 2023-10-26
Failed to get data for 2023-10-26: 429 Client Error: Too Many Requests for url: https://www.basketball-reference.com/friv/dailyleaders.cgi?month=10&day=26&year=2023
Getting box scores for 2023-10-27
Failed to get data for 2023-10-27: 429 Client Error: Too Many Requests for url: https://www.basketball-reference.com/friv/dailyleaders.cgi?month=10&day=27&year=2023
Getting box scores for 2023-10-28
Failed to get data for 2023-10-28: 429 Client Error: Too Many Requests for url: https://www.basketball-reference.com/friv/dailyleaders.cgi?month=10&day=28&year=2023
Getti

In [5]:
gamelog = leaguegamelog.LeagueGameLog(season='2024-25', player_or_team_abbreviation='P')
player_logs = gamelog.get_data_frames()[0]

print(player_logs[['PLAYER_NAME', 'GAME_DATE', 'TEAM_ABBREVIATION', 'MATCHUP', 'PTS']].head())

player_logs.to_csv("nba_player_box_scores_2023_24.csv", index=False)

          PLAYER_NAME   GAME_DATE TEAM_ABBREVIATION      MATCHUP  PTS
0      Xavier Tillman  2024-10-22               BOS  BOS vs. NYK    2
1  Karl-Anthony Towns  2024-10-22               NYK    NYK @ BOS   12
2        Jayson Tatum  2024-10-22               BOS  BOS vs. NYK   37
3       Derrick White  2024-10-22               BOS  BOS vs. NYK   24
4           Josh Hart  2024-10-22               NYK    NYK @ BOS   12


In [3]:
teamlog = leaguegamelog.LeagueGameLog(season='2023-24', player_or_team_abbreviation='T')  # 'T' = teams
team_logs = teamlog.get_data_frames()[0]

In [6]:
print(player_logs['GAME_DATE'].max())

2025-03-28


In [7]:
box_scores = pd.read_csv("NBA_Box_Score_24-25.csv")
player_avg = pd.read_csv("average_stats.csv")
team_perf = pd.read_csv("NBA_Team_Performance.csv")

In [9]:
box_scores['HOME'] = box_scores['MATCHUP'].str.contains(r'vs\.').astype(int)

box_scores['OPPONENT'] = box_scores['MATCHUP'].str.extract(r'vs\. (\w+)|@ (\w+)').bfill(axis=1).iloc[:, 0]

box_scores['TEAM'] = box_scores['TEAM_NAME'].str.upper().str.replace(" ", "_")
box_scores['OPPONENT'] = box_scores['OPPONENT'].str.upper()
box_scores['GAME_DATE'] = pd.to_datetime(box_scores['GAME_DATE'])

box_scores = box_scores.sort_values(['PLAYER_NAME', 'GAME_DATE'])

for stat in ['PTS', 'AST', 'REB']:
    box_scores[f'{stat}_ROLL3'] = box_scores.groupby('PLAYER_NAME')[stat].rolling(3, min_periods=1).mean().reset_index(0, drop=True)

In [10]:
player_avg.rename(columns={
    'name': 'PLAYER_NAME',
    'points_per_game': 'PPG',
    'assists_per_game': 'APG',
    'rebounds_per_game': 'RPG'
}, inplace=True)

full_df = box_scores.merge(player_avg, how='left', on='PLAYER_NAME')

In [ ]:
team_perf['TEAM'] = team_perf['TEAM'].str.upper().str.replace(" ", "_")
team_perf['OPPONENT'] = team_perf['OPPONENT'].str.upper().str.replace(" ", "_")

full_df = full_df.merge(
    team_perf[['TEAM', 'OPPONENT', 'TEAM_HOME', 'avg_points_scored', 'avg_points_allowed']],
    how='left',
    left_on=['TEAM', 'OPPONENT', 'HOME'],
    right_on=['TEAM', 'OPPONENT', 'TEAM_HOME']
)

In [12]:
full_df.to_csv("structured_betting_dataset.csv", index=False)

In [15]:
#Drop empty values in structured betting data
full_df[['avg_points_scored', 'avg_points_allowed']] = full_df[['avg_points_scored', 'avg_points_allowed']].fillna(-1)

In [16]:
full_df.to_csv("structured_betting_dataset.csv", index=False)

In [17]:
full_df['avg_points_scored'] = full_df['avg_points_scored'].fillna(-1.0)
full_df['avg_points_allowed'] = full_df['avg_points_allowed'].fillna(-1.0)

In [18]:
full_df.to_csv("structured_betting_dataset.csv", index=False)

In [19]:
full_df['MATCHUP_KNOWN'] = (full_df['avg_points_scored'] != -1.0).astype(int)

In [20]:
full_df.to_csv("structured_betting_dataset.csv", index=False)

In [ ]:

def r2_metric(y_true, y_pred):
    SS_res = np.sum(np.square(y_true - y_pred))
    SS_tot = np.sum(np.square(y_true - np.mean(y_true)))
    return 1 - SS_res / (SS_tot + 1e-7)

'''def lr_schedule(epoch, lr):
    if epoch <= 10:
        if epoch % 2 == 0 and epoch > 0:
            return lr * 0.9
    else:
        return lr * 0.9
    return lr'''


def build_rnn_sequences(df, player_name, stat, threshold, features, window=5):
    df_player = df[df['PLAYER_NAME'] == player_name].copy().sort_values("GAME_DATE")
    df_player['TARGET'] = (df_player[stat] > threshold).astype(int)

    scaler = StandardScaler()
    df_player[features] = scaler.fit_transform(df_player[features])

    X, y = [], []
    for i in range(len(df_player) - window):
        seq = df_player[features].iloc[i:i+window].values
        label = df_player['TARGET'].iloc[i+window]
        X.append(seq)
        y.append(label)

    return np.array(X), np.array(y), scaler

# LSTM Neural Network class
class NeuralNetwork:
    def __init__(self, X_train, X_test, y_train, y_test):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.model = None

    def build_model(self, input_shape):
        self.model = Sequential()
        self.model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
        self.model.add(LSTM(32, return_sequences=False))
        self.model.add(Dense(25, activation='relu'))
        self.model.add(Dense(1, activation='sigmoid'))

        self.model.compile(optimizer='adam',
                           loss='binary_crossentropy',
                           metrics=['accuracy', MeanAbsoluteError()])

        self.model.summary()

    def train(self, epochs=30, batch_size=32):
        #scheduler = LearningRateScheduler(lr_schedule)
        self.model.fit(self.X_train, self.y_train,
               validation_data=(self.X_test, self.y_test),
               epochs=epochs,
               batch_size=batch_size)
        self.model.save("rnn_player_model.h5")
        print("Model saved as 'rnn_player_model.h5'")

    def evaluate(self):
        results = self.model.evaluate(self.X_test, self.y_test)
        print(f"Loss: {results[0]}, Accuracy: {results[1]}, MAE: {results[2]}")
        
if __name__ == "__main__":

    df = pd.read_csv("structured_betting_dataset.csv")

    player = "Stephen Curry"
    stat = "AST"
    threshold = 4

    features = ['PTS_ROLL3', 'AST_ROLL3', 'REB_ROLL3', 'PPG', 'APG', 'RPG', 'HOME', 'avg_points_allowed']

    X, y, scaler = build_rnn_sequences(df, player, stat, threshold, features, window=5)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    nn = NeuralNetwork(X_train, X_test, y_train, y_test)
    nn.build_model(input_shape=(X.shape[1], X.shape[2]))
    nn.train()
    nn.evaluate()        


C:\Users\Brian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 5, 64)          │        18,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │           825 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,955 (124.82 KB)

 Trainable params: 31,955 (124.82 KB)

 Non-trainable params: 0 (0.00 B)

NameError: name 'scheduler' is not defined